In [1]:
# Importing useful libraries

from os.path import join
import time

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd

from sklearn import metrics
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils import shuffle

from sklearn.cluster import FeatureAgglomeration
from sklearn.feature_selection import RFE, GenericUnivariateSelect, chi2, mutual_info_classif, f_classif
from sklearn.model_selection import LeaveOneOut, KFold, RepeatedKFold

from neurocombat_sklearn import CombatModel

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tools import *

# Defining local variables
data_path = '/home/benoit/data/parotide/'

/home/benoit/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
df_meta = get_meta_data(data_path)

type_to_include=['t1', 't2', 'gado', 'diff']
# type_to_include=['t1']


ls_exams, id_to_feat, feat_to_id = load_features(df_meta, data_path, verbose=False, type_to_include=type_to_include)

features = []
labels = []

# Meta data fields to include in meta variables for harmonization

meta_fields = ['sexe', 'tesla', 'age']
meta_variables = []

for exam in ls_exams:
    lbl, ft, meta = format_exam(exam, feat_to_id, meta_fields=meta_fields)
    features.append(ft)
    labels.append(lbl)
    meta_variables.append(meta)
    
features = np.array(features)
labels = np.array(labels)
meta_variables = np.array(meta_variables)

print(f"{len(labels)} with {len(features[0])} features")
print(f"Labels : {len(labels) - sum(labels)} benign lesions and {sum(labels)} malignant")

# Adjusting the scales of the different features

scaler = StandardScaler()
rescaled_features = scaler.fit_transform(features)

print(f"Shape features : {rescaled_features.shape}")
print(f"Shape meta variables : {meta_variables.shape}")

n_features = features.shape[1]
n_exams = features.shape[0]

features = rescaled_features

107 with 432 features
Labels : 58 benign lesions and 49 malignant
Shape features : (107, 432)
Shape meta variables : (107, 3)


In [3]:
# Feature harmonization using ComBat

harmonization = CombatModel()

# h_sexe, h_age = None, None

h_tesla = meta_variables[:, 1].reshape(-1,1)
h_sexe = meta_variables[:, 0].reshape(-1,1)
h_age = meta_variables[:, 2].reshape(-1,1)

features = harmonization.fit_transform(features, h_tesla, h_sexe, h_age)

In [4]:
# K fold experiment

n_selected_features = 10
n_splits = 10
n_repeats = 10

classifier = LogisticRegression(penalty='l2', solver='liblinear', C=0.5)
# classifier = LogisticRegression(penalty='l1', solver='saga', C=1, l1_ratio=0)
# classifier = DecisionTreeClassifier()
# classifier = SVC(C=1, probability=True, kernel='rbf')
# classifier = RandomForestClassifier(n_estimators=20, verbose=0)


#iterator = KFold(n_splits=n_splits, random_state=None, shuffle=True)
iterator = RepeatedKFold(n_splits=n_splits, n_repeats=10, random_state=None)

features_summary = dict(zip([i for i in range(n_features)], np.zeros(n_features)))

ls_auc_train = np.zeros(n_splits*n_repeats) - 1
ls_auc_test = np.zeros(n_splits*n_repeats) - 1

ls_acc_train = np.zeros(n_splits*n_repeats) - 1
ls_acc_test = np.zeros(n_splits*n_repeats) - 1

mean_preds = np.zeros(len(features))

n = 0
for train_id, test_id in iterator.split(features):
    
    # Spliting between testing and training set
    train_features = features[train_id]
    train_labels = labels[train_id]

    test_labels = labels[test_id]
    
    if sum(test_labels) == 0 or sum(test_labels) == len(test_labels):
        print("Skipping this batch", end='\r')
        continue
    
    # Performing feature selection using RFE or p-value (on train set only)
    
    ###
    if False:
        feature_significance_p = feature_t_test(train_features, train_labels, id_to_feat)
        feature_significance_auc = feature_auc(train_features, train_labels, id_to_feat, LogisticRegression(penalty='none'))

        selection_p_value = choose_features_from_dict(feature_significance_p, n_selected_features, feat_to_id)
        
        selected_features_id = selection_p_value
    
    ###
    if False:
        estimator = classifier
        selector = RFE(estimator, n_selected_features, step=1).fit(train_features, train_labels)
        selection_rfe = np.where(selector.support_.astype(int) > 0)[0]
        
        selected_features_id = selection_rfe
        
        for i in selected_features_id:
            features_summary[i] += 1
    ###
    
    if True:
        agglo = FeatureAgglomeration(n_clusters=n_selected_features).fit(train_features)
        features_s = agglo.transform(features)
    
    # Applying feature selection to train and test subsets
    # features_s = features[:, selected_features_id]
    
    # Evaluation of the classifier using LOO strategy
    loo = LeaveOneOut()

    X = features_s
    y = labels
    
    clf = classifier
    # clf = SVC(C=0.8, probability=True, kernel='rbf')
    
    loo_predictions = np.zeros(len(X))
    loo_labels = np.zeros(len(X))
    
    for in_id, out_id in loo.split(X):

        X_train, X_test = X[in_id], X[out_id]
        y_train, y_test = y[in_id], y[out_id]

        exam = ls_exams[out_id[0]]

        clf.fit(X_train, y_train)

        pred = clf.predict_proba(X_test)[0, 1]

        loo_predictions[out_id[0]] = pred
        loo_labels[out_id[0]] = y_test
                        
        
        
    loo_predictions = np.array(loo_predictions)
    loo_labels = np.array(loo_labels)
                        
    mean_preds += loo_predictions
                        
    loo_predictions_b = loo_predictions > 0.5
    
    # Computing metrics on train and test set, separatly
    ls_auc_train[n] = metrics.roc_auc_score(loo_labels[train_id], loo_predictions[train_id])
    ls_auc_test[n] = metrics.roc_auc_score(loo_labels[test_id], loo_predictions[test_id])
    
    ls_acc_train[n] = metrics.accuracy_score(loo_labels[train_id], loo_predictions_b[train_id])
    ls_acc_test[n] = metrics.accuracy_score(loo_labels[test_id], loo_predictions_b[test_id])
                        
    print("K-fold {:2}/{:2} - Auc train : {:.3f} - Auc test: {:.3f} - Acc train : {:.3f} - Acc test: {:.3f}"
          .format(n+1, n_splits*n_repeats-1, ls_auc_train[n], ls_auc_test[n], ls_acc_train[n], ls_acc_test[n]) , end='\r')
                        
    n += 1

In [5]:
print("Average training AUC : {:.3f}".format(np.mean(ls_auc_train)))
print("Average testing AUC : {:.3f}\n\n-- -- -- --\n".format(np.mean(ls_auc_test)))

print("Average training ACC : {:.3f}".format(np.mean(ls_acc_train)))
print("Average testing ACC : {:.3f}".format(np.mean(ls_acc_test)))

Average training AUC : 0.644
Average testing AUC : 0.646

-- -- -- --

Average training ACC : 0.627
Average testing ACC : 0.642


In [6]:
m_p = mean_preds/n

for i in range(len(mean_preds)):
    with open(f'resultats.csv', "a+") as f:
        f.write(f'{ls_exams[i]["id"]},{m_p[i]},{labels[i]},{"train" if (i in train_id) else "test"}\n')

In [7]:
for i in order_dict(features_summary).keys():
    if features_summary[i] > 0:
        print(">{:40.40} - {:3}/100".format(id_to_feat[i], int(features_summary[i])))
    